<a href="https://colab.research.google.com/github/Amplil/laser/blob/master/laser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [LASER for NLP tasks - Part 2](https://www.engati.com/blog/laser-for-nlp-tasks-part-ii)のページを参考にしたプログラム

## インストール（1度goole driveに保存したら2回目以降は実行する必要ない。）

In [1]:
#!git clone https://github.com/facebookresearch/LASER

In [2]:
#cd "/content/drive/My Drive/Colab Notebooks/LASER_official"

In [3]:
#ls

In [4]:
#!export LASER="/content/drive/My Drive/Colab Notebooks/LASER"

In [5]:
#!export LASER="${HOME}/projects/laser"

In [6]:
#!bash ./install_models.sh

In [7]:
# !pip install --target "/content/drive/MyDrive/Colab Notebooks/packages" laserembeddings

In [8]:
# cd "/content/drive/MyDrive/Colab Notebooks/packages"

In [9]:
#!pip install laserembeddings

In [10]:
#!python -m laserembeddings download-models

In [11]:
#!python -m laserembeddings download-test-data

In [12]:
#!pytest tests/test_laser.py

In [13]:
#!pip install --target "/content/drive/MyDrive/Colab Notebooks/packages" laserembeddings[ja]

In [14]:
#!pip install --target "/content/drive/MyDrive/Colab Notebooks/packages" laserembeddings[zh]

In [15]:
#!pip install --target "/content/drive/MyDrive/Colab Notebooks/packages" boto3

In [34]:
#!pip install --target "/content/drive/MyDrive/Colab Notebooks/packages" laserembeddings

     |████████████████████████████████| 15.3MB 292kB/s 
     |████████████████████████████████| 860kB 51.8MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 804.1MB 21kB/s 
     |████████████████████████████████| 92kB 9.7MB/s 
     |████████████████████████████████| 307kB 58.3MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp37-none-any.whl size=883999 sha256=89f6ab06616c43df5a1c65d7bc650f2d37a858d2854581428a6c64b6ea13bf12
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 whic

In [32]:
#!pip install --target "/content/drive/MyDrive/Colab Notebooks/packages" pyforest

  Created wheel for pyforest: filename=pyforest-1.0.3-py2.py3-none-any.whl size=13716 sha256=102a4b88aba7fe9350f1741cca6cca2051b8e35973b38acc9f8f1ba577c250fa
  Stored in directory: /root/.cache/pip/wheels/af/0b/39/340a7f15fc8d4ff5ab50847b28789afea04452a9d51b9721b5
Successfully built pyforest


## インポート

In [17]:
"""
from google.colab import drive
drive.mount('/content/drive')
"""

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In [18]:
cd "/content/drive/My Drive/Colab Notebooks/data"

/content/drive/My Drive/Colab Notebooks/data


In [19]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/packages")

In [41]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
import re
import pandas as pd
from laserembeddings import Laser

## kaggleのコンペ[Tweet Sentiment Extractionのtest.csv](https://www.kaggle.com/c/tweet-sentiment-extraction/data?select=test.csv)を使う

In [21]:
#df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/tweet_train.csv', index_col=0).fillna('')
#df_train

In [22]:
%ls

1987.csv                     20210120_acc_geo_data_fix.csv
1988.csv                     20210120_geomagnetism_2.csv
1989.csv                     20210208_acceleration.csv
20210106_acceleration.csv    20210208_geomagnetism.csv
20210106_geomagnetism.csv    20210208_pressure.csv
20210118_acceleration-2.csv  training.1600000.processed.noemoticon.csv
20210118_geomagnetism-2.csv  tweet-sentiment-test.csv
20210118_pressure-2.csv      tweet_train.csv
20210120_acceleration_2.csv


In [23]:
df_train = pd.read_csv('tweet-sentiment-test.csv', index_col=0).fillna('')
df_train

,text,sentiment
textID,,
f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
96d74cb729,Shanghai is also really exciting (precisely -...,positive
eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
01082688c6,happy bday!,positive
33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive
...,...,...
e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep ...",negative
416863ce47,All alone in this old house again. Thanks for...,positive
6332da480c,I know what you mean. My little dog is sinkin...,negative


In [24]:
df_train.shape

(3534, 2)

In [25]:
#!pip install icecream

In [26]:
#feature = df_train.text # 特徴データ
#target = df_train.sentiment # ラベルデータ

from icecream import ic
#datax = df_train["selected_text"]
datax = df_train["text"]

datay = df_train["sentiment"]
ic(datax.shape)
ic(datay.shape)

ic| datax.shape: (3534,)
ic| datay.shape: (3534,)


(3534,)

In [27]:
#scikit-learnを用いたデータ分割
from sklearn.model_selection import train_test_split
trainx, testx, trainy, testy = train_test_split(datax, datay, test_size=0.3)
#X1, X2, Y1, Y2 = train_test_split(feature, target, test_size=0.3)
ic(trainx.shape)
ic(testx.shape)

ic| trainx.shape: (2473,)
ic| testx.shape: (1061,)


(1061,)

In [ ]:
trainx

textID
09140327f6    tomorrow is going to be sooo awkward & embaras...
1aa65233b0    Enjoying the cool breeze at the beach. It help...
7993d38b85     ha! lemme know what you find (tho I suspect i...
9e8665abc5    Dang  I left my Rolando`s dinner leftovers at ...
60e5c2c335           chillin bored drinking a margarita. txt me
                                    ...                        
5f455a3147                                       Flat tire city
eb2fc4d281     haaaw..well i get out of class at 10:50..i ho...
d8c30b8a27                  _carter The video is set to private
79852f9590     err i think they`re like 15 bucks a ticket ca...
0cc4081476    lol haha very funnyyy lol im talking to my bes...
Name: text, Length: 2473, dtype: object

In [ ]:
trainy

textID
09140327f6     neutral
1aa65233b0    positive
7993d38b85    positive
9e8665abc5     neutral
60e5c2c335    negative
                ...   
5f455a3147     neutral
eb2fc4d281    positive
d8c30b8a27     neutral
79852f9590    negative
0cc4081476     neutral
Name: sentiment, Length: 2473, dtype: object

In [28]:
#!pip install pyforest

In [36]:
#!python -m laserembeddings download-models

/usr/bin/python3: No module named laserembeddings


## embeddings

In [37]:
from laserembeddings import Laser
import pyforest
trainx_tokenization = np.array(Laser().embed_sentences(trainx, lang='en'))
testx_tokenization = np.array(Laser().embed_sentences(testx, lang='en'))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
ic(trainx_tokenization.shape)
ic(testx_tokenization.shape)

ic| trainx_tokenization.shape: (2473, 1024)
ic| testx_tokenization.shape: (1061, 1024)


(1061, 1024)

## 例

In [50]:
from sklearn.linear_model import LogisticRegression
#model_test = sklearn.linear_model.LogisticRegression()
model_test =LogisticRegression()
model_test.fit(trainx_tokenization, trainy)
trainy_pred = model_test.predict(trainx_tokenization)
testy_pred = model_test.predict(testx_tokenization)
ic(testy_pred[:5])
print("accuracy on training data:",sklearn.metrics.accuracy_score(trainy, trainy_pred))
print("accuracy on testing data:",sklearn.metrics.accuracy_score(testy, testy_pred))
print("classification report on testing data:")
print(sklearn.metrics.classification_report(testy,testy_pred))

ic| testy_pred[:5]: array(['neutral', 'neutral', 'neutral', 'neutral', 'negative'],
                          dtype=object)


accuracy on training data: 0.7205822887181561
accuracy on testing data: 0.6550424128180962
classification report on testing data:
              precision    recall  f1-score   support

    negative       0.68      0.54      0.60       308
     neutral       0.59      0.74      0.66       418
    positive       0.75      0.65      0.70       335

    accuracy                           0.66      1061
   macro avg       0.67      0.64      0.65      1061
weighted avg       0.67      0.66      0.65      1061



## Keras deeplearning

In [ ]:
# trainy_ = np.array(trainy)
# ic(trainy_.shape)

In [51]:
ic(type(trainy))
# trainy.loc[trainy['sentiment']== 'positive', 'sentiment']=2
values = np.array(trainy)
print(values)
label_encoder = sklearn.preprocessing.LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
onehot_encoder = sklearn.preprocessing.OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
# inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
# print(inverted)

ic| type(trainy): <class 'pandas.core.series.Series'>


<IPython.core.display.Javascript object>

['neutral' 'positive' 'positive' ... 'neutral' 'negative' 'neutral']
[1 2 2 ... 1 0 1]
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


In [52]:
def oh_ecoder(x):
  values = np.array(x)
  # print(values)
  label_encoder = sklearn.preprocessing.LabelEncoder()
  integer_encoded = label_encoder.fit_transform(values)
  # print(integer_encoded)
  onehot_encoder = sklearn.preprocessing.OneHotEncoder(sparse=False)
  integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
  onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
  # print(onehot_encoded)
  return onehot_encoded

In [53]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape= (1024,)))
model.add(tf.keras.layers.Dense(512, activation="relu"))
model.add(tf.keras.layers.Dense(512, activation="relu"))
model.add(tf.keras.layers.Dense(512, activation="relu"))
model.add(tf.keras.layers.Dense(512, activation="relu"))
model.add(tf.keras.layers.Dense(512, activation="relu"))
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(3, activation="softmax"))
model.compile(loss= tf.keras.losses.categorical_crossentropy,
       optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5),
       metrics = ["accuracy"]
    )
model.fit(trainx_tokenization, ic(oh_ecoder(trainy)), batch_size= 32, epochs= 50, callbacks=[])

<IPython.core.display.Javascript object>

ic| oh_ecoder(trainy): array([[0., 1., 0.],
                              [0., 0., 1.],
                              [0., 0., 1.],
                              ...,
                              [0., 1., 0.],
                              [1., 0., 0.],
                              [0., 1., 0.]])


Epoch 1/50
78/78 [==============================] - 2s 19ms/step - loss: 1.0979 - accuracy: 0.3964
Epoch 2/50
78/78 [==============================] - 1s 19ms/step - loss: 1.0950 - accuracy: 0.4193
Epoch 3/50
78/78 [==============================] - 1s 19ms/step - loss: 1.0922 - accuracy: 0.4023
Epoch 4/50
78/78 [==============================] - 1s 19ms/step - loss: 1.0861 - accuracy: 0.4056
Epoch 5/50
78/78 [==============================] - 1s 19ms/step - loss: 1.0777 - accuracy: 0.4076
Epoch 6/50
78/78 [==============================] - 1s 18ms/step - loss: 1.0667 - accuracy: 0.4166
Epoch 7/50
78/78 [==============================] - 1s 18ms/step - loss: 1.0558 - accuracy: 0.4003
Epoch 8/50
78/78 [==============================] - 1s 19ms/step - loss: 1.0285 - accuracy: 0.4031
Epoch 9/50
78/78 [==============================] - 1s 19ms/step - loss: 0.9800 - accuracy: 0.4410
Epoch 10/50
78/78 [==============================] - 1s 19ms/step - loss: 0.9194 - accuracy: 0.5714
Epoch 11/

In [54]:
print("accuracy on training data:", model.evaluate(trainx_tokenization, oh_ecoder(trainy)))
print("accuracy on validation data:",model.evaluate(testx_tokenization, oh_ecoder (testy)))

<IPython.core.display.Javascript object>

78/78 [==============================] - 1s 6ms/step - loss: 0.1845 - accuracy: 0.9543
accuracy on training data: [0.18446016311645508, 0.9543064832687378]


<IPython.core.display.Javascript object>

34/34 [==============================] - 0s 6ms/step - loss: 0.9710 - accuracy: 0.6758
accuracy on validation data: [0.9710211157798767, 0.6757775545120239]


In [55]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_4 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               3

In [81]:
model.save('laser_model.h5')

In [82]:
ls

1987.csv                     20210120_acc_geo_data_fix.csv
1988.csv                     20210120_geomagnetism_2.csv
1989.csv                     20210208_acceleration.csv
20210106_acceleration.csv    20210208_geomagnetism.csv
20210106_geomagnetism.csv    20210208_pressure.csv
20210118_acceleration-2.csv  laser_model.h5
20210118_geomagnetism-2.csv  training.1600000.processed.noemoticon.csv
20210118_pressure-2.csv      tweet-sentiment-test.csv
20210120_acceleration_2.csv  tweet_train.csv


# [Multilingual Sentence Models in NLP](https://towardsdatascience.com/multilingual-sentence-models-in-nlp-476f1f246d2f)のページを参考にしたプログラム

In [85]:
import requests
import numpy as np
def get_vect(query_in, lang = 'en', address = '127.0.0.1:8050'):
    url = "http://" + address + "/vectorize"
    params = {"q": query_in, "lang": lang}
    resp = requests.get(url=url, params=params).json()
    return resp["embedding"]

In [86]:
input_dict = {
    "en":"Machine learning isn't as hard as people think.",
    "de":"Maschinelles Lernen ist nicht so schwer wie die Leute denken.",
    "fr":"L'apprentissage automatique n'est pas aussi difficile qu'on le pense.",
    "it":"L'apprendimento automatico non è così difficile come la gente pensa.",
    "es":"El aprendizaje automático no es tan difícil como la gente piensa.",
    "ru":"Машинное обучение не так сложно, как думают люди.",
    "ar":"التعلم الآلي ليس صعبا كما يظن الناس.",
    "pt":"O aprendizado de máquina não é tão difícil quanto as pessoas pensam.",
}

In [87]:
embeddings = laser.embed_sentences(input_dict,
    lang='en')  # lang is only used for tokenization
embeddings

array([[ 1.73062121e-03,  1.61062065e-03,  6.12360600e-04, ...,
         1.26744648e-02, -1.58457295e-03,  1.13074956e-02],
       [ 9.65316940e-05, -3.28753740e-05,  6.11302268e-04, ...,
         1.05216103e-02, -1.96547760e-03,  8.63582827e-03],
       [ 4.03991388e-03, -1.91512154e-05,  3.35007068e-03, ...,
        -1.11156485e-04, -5.82172116e-03,  1.38939777e-02],
       ...,
       [ 8.72488308e-04, -4.55544825e-04,  1.17029808e-03, ...,
         1.83722395e-02, -4.77603823e-03, -8.90916958e-03],
       [ 3.67702474e-03,  3.21718701e-03,  5.37057116e-04, ...,
         1.25199193e-02, -2.54276581e-03, -2.41810107e-03],
       [ 7.04408111e-03,  1.12752290e-03,  1.12105464e-03, ...,
         2.39676237e-02, -2.89872428e-03,  1.54712396e-02]], dtype=float32)

In [88]:
embedded_dict = {}
for key in input_dict:
    embedded_dict[key] = np.array(get_vect(input_dict[key], lang = key))

ConnectionError: ignored

In [ ]:
embeddings[0]

array([ 0.00173062,  0.00161062,  0.00061236, ...,  0.01267446,
       -0.00158457,  0.0113075 ], dtype=float32)

In [ ]:
embeddings[0].shape

(1024,)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_lib = cosine_similarity(embedded_dict['it'], embedded_dict['es'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_lib = cosine_similarity(embeddings[0], embeddings[1])

ValueError: ignored

# [laserembeddings · PyPI](https://pypi.org/project/laserembeddings/)のページを参考にしたプログラム

In [73]:
from laserembeddings import Laser
laser = Laser()
# if all sentences are in the same language:
embeddings = laser.embed_sentences(
    ['let your neural network be polyglot',
     'use multilingual embeddings!'],
    lang='en')  # lang is only used for tokenization
# embeddings is a N*1024 (N = number of sentences) NumPy array
embeddings

array([[0.00156426, 0.01067378, 0.00388247, ..., 0.021797  , 0.0062856 ,
        0.01434554],
       [0.01456157, 0.00154333, 0.00090701, ..., 0.01484861, 0.00712931,
        0.0476219 ]], dtype=float32)

In [ ]:
embeddings = laser.embed_sentences(
    ['I love pasta.',
     "J'adore les pâtes.",
     'Ich liebe Pasta.'],
    lang=['en', 'fr', 'de'])

In [ ]:
%%bash
# DOCS: https://github.com/facebookresearch/LASER/blob/master/install_models.sh

mkdir -p models/laser/
# for FILE in bilstm.eparl21.2018-11-19.pt eparl21.fcodes eparl21.fvocab bilstm.93langs.2018-12-26.pt 93langs.fcodes 93langs.fvocab; do
for FILE in bilstm.93langs.2018-12-26.pt 93langs.fcodes 93langs.fvocab; do
    wget -cq https://dl.fbaipublicfiles.com/laser/models/$FILE -O models/laser/$FILE
done

In [ ]:
from laserembeddings import Laser

path_to_bpe_codes = "./models/laser/93langs.fcodes"
path_to_bpe_vocab = "./models/laser/93langs.fvocab"
path_to_encoder ="./models/laser/bilstm.93langs.2018-12-26.pt"

laser = Laser(path_to_bpe_codes, path_to_bpe_vocab, path_to_encoder)

# you can also supply file objects instead of file paths

In [ ]:
from io import BytesIO, StringIO
from laserembeddings import Laser
import boto3

s3 = boto3.resource('s3')
MODELS_BUCKET = ...

f_bpe_codes = StringIO(s3.Object(MODELS_BUCKET, path_to_bpe_codes).get()['Body'].read().decode('utf-8'))
f_bpe_vocab = StringIO(s3.Object(MODELS_BUCKET, path_to_bpe_vocab).get()['Body'].read().decode('utf-8'))
f_encoder = BytesIO(s3.Object(MODELS_BUCKET, path_to_encoder).get()['Body'].read())

laser = Laser(f_bpe_codes, f_bpe_vocab, f_encoder)